In [80]:
#!pip install beautifulsoup
#!wget https://dumps.wikimedia.org/enwiki/20190420/enwiki-20190420-pages-articles-multistream1.xml-p10p30302.bz2
#!bunzip2 enwiki-20190420-pages-articles-multistream1.xml-p10p30302.bz2    

In [81]:
# Count lines, words, characters in XML data file
!wc enwiki-20190420-pages-articles-multistream1.xml-p10p30302

 4586335 74563548 642819893 enwiki-20190420-pages-articles-multistream1.xml-p10p30302


In [82]:
from bs4 import BeautifulSoup
import pandas as pd

In [83]:
def get_pages_from_xml_file(filename, start_tag='<page>', end_tag='</page>'):
    """Yields each page from the specified XML data file."""
    page = None
    with open(filename) as f:
        for line in f:
            if start_tag in line:
                page = []
                page.append(line)
            elif end_tag in line:
                page.append(line)
                page_xml = ''.join(page)
                yield page_xml
                page = None
            else:
                if page is not None:
                    page.append(line)

In [119]:
filename = 'enwiki-20190420-pages-articles-multistream1.xml-p10p30302'
pages = list(get_pages_from_xml_file(filename))

In [120]:
def get_title_from_page_xml(page_xml):
    """Returns the title of the given page."""
    soup = BeautifulSoup(page_xml, 'lxml')
    return soup.select_one('title').text

In [121]:
len(pages)  # Show the number of pages

19822

In [122]:
%time titles = [get_title_from_page_xml(page) for page in pages]

CPU times: user 57.2 s, sys: 4.38 s, total: 1min 1s
Wall time: 1min 5s


In [123]:
titles[:5] + ['...'] + titles[-5:]  # Show the first and last 5 titles

['AccessibleComputing',
 'Anarchism',
 'AfghanistanHistory',
 'AfghanistanGeography',
 'AfghanistanPeople',
 '...',
 'The Lord of the Rings/One Ring',
 'Tax Freedom Day',
 'Tax',
 'Transhumanism',
 'TARDIS']

In [124]:
print(pages[0])  # Print the first page

  <page>
    <title>AccessibleComputing</title>
    <ns>0</ns>
    <id>10</id>
    <redirect title="Computer accessibility" />
    <revision>
      <id>854851586</id>
      <parentid>834079434</parentid>
      <timestamp>2018-08-14T06:47:24Z</timestamp>
      <contributor>
        <username>Godsy</username>
        <id>23257138</id>
      </contributor>
      <comment>remove from category for seeking instructions on rcats</comment>
      <model>wikitext</model>
      <format>text/x-wiki</format>
      <text xml:space="preserve">#REDIRECT [[Computer accessibility]]

{{R from move}}
{{R from CamelCase}}
{{R unprintworthy}}</text>
      <sha1>42l0cvblwtb4nnupxm6wo000d27t6kf</sha1>
    </revision>
  </page>



In [125]:
def get_text_from_page_xml(page):
    """Returns the text of the given page."""
    soup = BeautifulSoup(page, 'lxml')
    return soup.select_one('text').text

In [126]:
page = pages[0]
soup = BeautifulSoup(page, 'lxml')
text = get_text_from_page_xml(page)
text, type(text)

('#REDIRECT [[Computer accessibility]]\n\n{{R from move}}\n{{R from CamelCase}}\n{{R unprintworthy}}',
 str)

In [127]:
data = []
for page in pages:
    row = {
        'title': get_title_from_page_xml(page),
        'text': get_text_from_page_xml(page),
    }
    data.append(row)

In [128]:
df = pd.DataFrame(data)
df.head()

,text,title
0,#REDIRECT [[Computer accessibility]]\n\n{{R fr...,AccessibleComputing
1,{{redirect2|Anarchist|Anarchists|the fictional...,Anarchism
2,#REDIRECT [[History of Afghanistan]]\n\n{{Redi...,AfghanistanHistory
3,#REDIRECT [[Geography of Afghanistan]]\n\n{{Re...,AfghanistanGeography
4,#REDIRECT [[Demographics of Afghanistan]]\n\n{...,AfghanistanPeople


In [129]:
text_length = []
for row in range(len(df.text)):
    text_length.append(len(df.text[row]))

In [130]:
df['text_length'] = 'text_length'

In [131]:
df.columns

Index(['text', 'title', 'text_length'], dtype='object')

In [132]:
for row in range(len(df)):
    df.text_length[row] = text_length[row]

In [133]:
high_values = df.text_length.sort_values(ascending=False).head().index

In [134]:
df.loc[high_values]

,text,title,text_length
10165,{{short description|Wikipedia list page}}\n{{s...,List of compositions by Johann Sebastian Bach,432964
15129,{{about|the country}}\n{{pp-semi-indef}}\n{{pp...,Pakistan,376957
15287,{{Redirect|Philippine|the town in the Netherla...,Philippines,357950
9299,{{Short description|Overview of relations}}{{U...,Foreign relations of India,317725
8838,{{short description|Aspect of history}}\n{{pp-...,History of India,316816


### Challenge

#### 1. Create a Pandas dataframe containing the title and text of each page.

* Implement the `get_text_from_page_xml` function above.
* Re-create the dataframe with the text field filled in.

#### 2. Identify the five pages that have the _longest_ text.

* Find the length of each page's `<text>...</text>` element and add it to your dataframe.
* Sort the data frame by text length, descending.
* What are the titles of the five longest articles?

# cleaning up the wiki text

In [135]:
#messy and hard to read
df.text[1][:1000]

"{{redirect2|Anarchist|Anarchists|the fictional character|Anarchist (comics)|other uses|Anarchists (disambiguation)}}\n{{pp-move-indef}}{{short description|Political philosophy that advocates self-governed societies}}\n{{use dmy dates|date=July 2018}}\n{{use British English|date=January 2014}}\n{{anarchism sidebar}}\n{{basic forms of government}}\n'''Anarchism''' is an [[anti-authoritarian]] [[political philosophy]]{{sfnm|1a1=McLaughlin|1y=2007|1p=59|2a1=Flint|2y=2009|2p=27}} that advocates [[Self-governance|self-governed]] societies based on voluntary, [[cooperative]] institutions and the rejection of [[Hierarchy|hierarchies]] those societies view as unjust. These institutions are often described as [[Stateless society|stateless societies]],{{sfnm|1a1=Craig|1y=2005|1p=14|2a1=Sheehan|2y=2003|2p=85}} although several authors have defined them more specifically as distinct institutions based on non-hierarchical or [[Free association (communism and anarchism)|free associations]].{{sfn|Sui

In [136]:
#bring in a parsing library
import wikitextparser as wtp

In [137]:
parsed = wtp.parse(df.text[1])
to_remove = parsed.templates

In [138]:
#look for the wikitext templates and remove them
#from the text
for index in range(len(to_remove)):
    if str(to_remove[index]) in df.text[1]:
        df.text[1] = df.text[1].replace(f'{to_remove[index]}', "")

In [139]:
#much easier to read
df.text[1][:1000]

"\n\n\n\n\n\n'''Anarchism''' is an [[anti-authoritarian]] [[political philosophy]] that advocates [[Self-governance|self-governed]] societies based on voluntary, [[cooperative]] institutions and the rejection of [[Hierarchy|hierarchies]] those societies view as unjust. These institutions are often described as [[Stateless society|stateless societies]], although several authors have defined them more specifically as distinct institutions based on non-hierarchical or [[Free association (communism and anarchism)|free associations]]. Anarchism holds the [[Sovereign state|state]] to be undesirable, unnecessary and harmful.\n\nAnarchism is often considered a [[Far-left politics|far-left]]   ideology and much of its [[Anarchist economics|economics]] and [[Anarchist law|legal philosophy]] reflect [[Libertarian socialism|anti-authoritarian interpretations]] of [[Anarcho-communism|communism]], [[Collectivist anarchism|collectivism]], [[Anarcho-syndicalism|syndicalism]], [[Mutualism (economic the